In [188]:
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [189]:
df1 = pd.read_csv("/Users/monishakrothapalli/Documents/GitHub/playlistGen/spotify_data.csv")

In [191]:
df1.shape

(1159764, 20)

In [190]:
df1.info()
df1.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159764 entries, 0 to 1159763
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Unnamed: 0        1159764 non-null  int64  
 1   artist_name       1159749 non-null  object 
 2   track_name        1159763 non-null  object 
 3   track_id          1159764 non-null  object 
 4   popularity        1159764 non-null  int64  
 5   year              1159764 non-null  int64  
 6   genre             1159764 non-null  object 
 7   danceability      1159764 non-null  float64
 8   energy            1159764 non-null  float64
 9   key               1159764 non-null  int64  
 10  loudness          1159764 non-null  float64
 11  mode              1159764 non-null  int64  
 12  speechiness       1159764 non-null  float64
 13  acousticness      1159764 non-null  float64
 14  instrumentalness  1159764 non-null  float64
 15  liveness          1159764 non-null  float64
 16  

Unnamed: 0           0
artist_name         15
track_name           1
track_id             0
popularity           0
year                 0
genre                0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
duration_ms          0
time_signature       0
dtype: int64

In [192]:
from sklearn.preprocessing import StandardScaler

In [219]:
def section_data(genre):
    sampled = df1[df1['genre'] == genre]
    features = ['valence', 'energy', 'danceability', 'loudness']
    df2 = sampled[features]
    scaler = StandardScaler()
    df2  = scaler.fit_transform(df2)
    sim_matrix = cosine_similarity(df2)
    return sim_matrix, sampled 

In [205]:
def get_indices(genre):
    _, sampled = section_data(genre)
    indices = {song: i for i, song in enumerate(sampled['track_name'])}
    return indices; 

In [206]:
def get_genre(song, artist):
    genre = df1.loc[(df1['track_name'] == song) & (df1['artist_name'] == artist), 'genre'].values[0]
    return genre 

In [220]:
def get_recs(song, aritst, length):
    genre = get_genre(song, aritst)

    cosine_sim, sampled = section_data(genre)
    indices = get_indices(genre)

    if song not in indices:
        raise ValueError("Song '{song}' not found.")
    
    index = indices[song]

    sim_scores = list(enumerate(cosine_sim[index]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[0:length]

    song_index = [i[0] for i in sim_scores]

    return sampled['artist_name'].iloc[song_index],sampled['track_name'].iloc[song_index] 

In [222]:
artists, tracks = get_recs('Kill Bill', 'SZA', 20)
print(tracks)
print(artists)

569184                                 Kill Bill
515568                                     Ghost
515610                               My Universe
515584    Peaches (feat. Daniel Caesar & Giveon)
739350             She Will Be Loved - Radio Mix
461999                            Shinunoga E-Wa
93573                              We Can't Stop
612512                                     River
462007                                    Maniac
569187                                 Anti-Hero
146786                                Like I Can
612580           Deserve Me (with Summer Walker)
569254                               All My Love
293484                          ...Ready For It?
515577         Wildest Dreams (Taylor's Version)
93593                                  Right Now
569202                            Seek & Destroy
406392                                   ROXANNE
293526                                   Go Gina
515595     You Belong With Me (Taylor’s Version)
Name: track_name, dt